### Model and Prompt

`langchain`은 50개 이상의 서드파티 공급업체 및 플랫폼과의 통합을 제공하며, OpenAI, Azure OpenAI, Databricks, MosaicML은 물론 허깅페이스 허브 및 오픈소스 LLM 세계와의 통합도 지원한다. 

In [1]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [2]:
from langchain_openai import OpenAI
llm = OpenAI(
    openai_api_key = openai.api_key, 
    max_tokens = 1024
)

In [4]:
print(llm.invoke('2026년 가격이 상승할 것으로 기대되는 미국과 한국 증시의 섹터나 테마는 뭘까? 각각 3개만 알려줘'))



미국 증시:
1. 기술 섹터 - 인공지능, 클라우드 컴퓨팅, 사물인터넷 등 미래 성장 가능성이 높은 기술 기업들이 포함되어 있어서 2026년에도 높은 수익을 기대할 수 있음.
2. 의료 섹터 - 고령화 진행으로 인해 의료 관련 기업들의 수요가 계속 증가할 것으로 예상되어 있으며, 코로나19 팬데믹 이후 의료 관련 기업들의 기술 혁신에 대한 관심이 높아져 더욱 성장할 수 있음.
3. 소비재 섹터 - 경기 회복에 따른 소비 증가로 인해 소비재 기업들의 수익이 증가할 것으로 예상됨. 특히, 온라인 쇼핑과 배달 서비스 등 디지털 소비 트렌드에 대한 기업들의 성장 가능성이 높음.

한국 증시:
1. 5G 관련 기업 - 5G 기술의 확대로 인해 이를 지원하는 기업들의 수익도 증가할 것으로 예상됨. 특히, 네트워크 인프라와 관련된 기업들의 성장 가능성이 높음.
2. 바이오 의약품 - 코로나19 팬데믹 이후 바이오 의약품 기업들의 기술력과 수요가 높아져 성장 가능성이 큼. 또한, 고령화 진행에 따른 의료 분야의 성장도 바이오 의약품 기업들에게 긍정적인 영향을 줄 것으로 예상됨.
3. 신재생 에너지 - 기후 변화 문제로 인해 신재생 에너지에 대한 관심이 높아져 성장 가능성이 큼. 특히, 정부의 신재생 에너지 정책 지원으로 인해 관련 기업들의 성장이 기대됨.


#### prompt template

prompt template은 언어 모델에 대한 프롬프트를 생성하는 방법을 정의하는 구성 요소이다. 여기에는 변수, placeholder, prefix, suffix, 그리고 데이터와 과업에 따라 맞춤화할 수 있는 기타 요소가 포함될 수 있다.
예를 들어, 언어 모델을 사용하여 한 언어에서 다른 언어로 번역을 생성하고 싶다고 가정해 보자. 다음과 같은 prompt template을 사용할 수 있다.

```
sentence : {sentence}
translate to {language} :
```

이 템플릿은 다음과 같이 쉽게 구현할 수 있다.

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

template = """
문장 : {sentence}
{language}로 번역 :
"""

prompt = ChatPromptTemplate.from_template(
    template = template,
)

chain = prompt | llm

result = chain.invoke({
    "sentence": "Hello world",
    "language": "한국어"
})

In [10]:
print(result.content)

안녕하세요, 세상!


**참고**
LLM으로 예제를 실행하는 동안 모델 자체의 확률성으로 인해 출력은 실행할 때마다 달라진다. 출력의 변동 폭을 줄이기 위해서는 `temperature` parameter를 조정하여 모델을 보다 결정론적으로 만들 수 있다. 이 때 parameter의 범위는 0(결정론적)에서 1(확률론적)까지이다.

- completion model : 텍스트 입력을 받아 텍스트 출력을 생성하는 LLM의 한 유형이다. 과업과 훈련한 데이터에 따라 일관되고 관련성 있는 방식으로 prompt를 이어가려고 노력한다. 예를 들어, 완성 모델을 프롬프트에 따라서 요약, 번역, 스토리, 코드, 가사 등을 생성할 수 있다.
- chat model : 대화 응답을 생성하도록 설계된 특수한 종류의 완성 모델이다. 메세지 목록을 입력으로 받으며, 각 메세지에는 역할(시스템, 사용자 또는 어시스턴트)과 콘텐츠가 있다. 채팅 모델은 이전 메세지와 시스템 지시를 기반으로 어시스턴트 역할에 대한 새 메세지를 생성하려고 시도한다.

완성모델과 채팅 모델의 주요 차이점은 완성 모델을 하나의 텍스트 입력을 프롬프트로 기대하는 반면, 채팅 모델은 메세지 목록을 입력으로 기대한다는 점이다.